<a href="https://colab.research.google.com/github/MatteoRigoni/MachineLearningPlayground/blob/master/BaseMachineLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Tecniche machine learning

> Apprendimento supervisionato (abbiamo input e output)
*  Regressione (l'output è un valore continuo, una quantità)
*  Classificazione (l'output è un valore discreto)

> Apprendimento non supervisionato (abbiamo solo input)
*  Associations (trovare regole che descrivono una porzione grande di dati)
*  Clustering (si raggruppano i dati per proprietà comuni)

> Apprendimento semi-supervisionato
*  Utilizzo di molti esempi senza label e pochi esempi con label

> Apprendimento per rinforzo
*  Si basa sulla realizzazione di **agenti intelligenti** in grado di prendere decisioni ed eseguire **azioni** in uno specifico **ambiente** al fine di massimizzare un **reward** e raggiungere un **obiettivo**


## Dataset

In [ ]:
import pandas as pd

In [ ]:
BASE_URL = "https://raw.githubusercontent.com/ProfAI/machine-learning-fondamenti/main/datasets/"

*csv con indicazione colonna indice*



In [ ]:
df = pd.read_csv(BASE_URL + "shirts_example.csv", index_col=0)
df.head()

*tsv con indicazione colonna indice*

In [ ]:
df = pd.read_csv(BASE_URL + "shirts_example.tsv", index_col=0, sep="\t")
df.head()

*json strutturato in formato tabellare*

In [ ]:
df = pd.read_json(BASE_URL + "shirts_example.json")
df.head(10)

*xml con elementi "row" indicati esplicitamente*

In [ ]:
df = pd.read_xml(BASE_URL + "shirts_example.xml", xpath='.//row')
df.head()

*html con unica table con indicazione indice e riga di header*

In [ ]:
df = pd.read_html(BASE_URL + "shirts_example.html", index_col=0, header = 0)
type(df) # lista di tutte le tabelle
type(df[0])
df[0].head()

*excel office oppure openoffice*

In [ ]:
pip install odfpy

In [ ]:
df = pd.read_excel(BASE_URL + "shirts_example.ods", index_col=0) #header=None per non usare prima riga come header
df.head()

## Data Preprocessing

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
BASE_URL = "https://raw.githubusercontent.com/ProfAI/machine-learning-fondamenti/main/datasets/"

*Ordinal encoding su variabile qualitativa*

Assegnazione di valore numero ordinato da 1

*  pandas

In [ ]:
df = pd.read_csv(BASE_URL+"shirts.csv", index_col=0)

size_mapping = {"S":1, "M":2, "L":3, "XL":4}
type(df["taglia"] ) #series of dataframe
df["taglia"] = df["taglia"].map(size_mapping)
df.head()

*  numpy

In [ ]:
df = pd.read_csv(BASE_URL+"shirts.csv", index_col=0)

X = df.values #array numpy multidimensionale (100x3)
X.shape

size_mapping = {"S":1, "M":2, "L":3, "XL":4}
fmap = np.vectorize(lambda t:size_mapping[t]) #funzione vettorizzata da applicare ad array numpy
X[:,0] = fmap(X[:,0])
X[:5,:]

*One hot encoding su variabili categoriche*

Aggiunta di colonna true/false per ogni possibile valore

*  pandas

In [ ]:
df = pd.read_csv(BASE_URL+"shirts.csv", index_col=0)

df = pd.get_dummies(df, columns=["colore"], prefix="color", prefix_sep="-") # "nomeColonnaOriginale_Classe" di default
df.head()

*  numpy con scikit-learn

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

df = pd.read_csv(BASE_URL+"shirts.csv", index_col=0)

X = df.values #array numpy multidimensionale (100x3)
X.shape

transf = ColumnTransformer(
    [
        ("ohe", OneHotEncoder(), [1]) #tupla identificativa, ultimo parametro è la lista delle colonne
    ],
    remainder = "passthrough" #per mantenere colonne non modificate
)

X = transf.fit_transform(X) #array numpy trasformato
X

*Label encoding*

Codificare variabile target che si presenta come stringa


*   pandas (come ordinal encoding sopra, assegnando valori 1 e 0)

In [ ]:
df = pd.read_csv(BASE_URL+"shirts_sold.csv", index_col=0)

size_mapping = {"SI":1, "NO":0}
df["venduta"] = df["venduta"].map(size_mapping)
df.head()

*  numpy

In [ ]:
df = pd.read_csv(BASE_URL+"shirts_sold.csv", index_col=0)
df.head()

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df["venduta"] = le.fit_transform(df["venduta"])
print(df.head())

le.classes_ #classi usate

# è possibile applicare trasformazione inversa...
y = [0,1,0,0,1]
y = le.inverse_transform(y)
print(y)

*Drop per dati mancanti*

In [ ]:
df = pd.read_csv(BASE_URL+"iris_missing.csv", index_col=0)
df.head()

df.count() # numero colonne mancano su sepal_width e petal_width
print(df.isna().sum()) # numero valori NA per colonna
print(df.shape) # numero osservazioni iniziale

#rimozione per riga...
df_drop = df.copy()
df_drop = df_drop.dropna()
df_drop.head()
print(df_drop.shape)

# - usare "how" = all per rimuovere righe con tutte colonne vuote...
# - usare "subset" = ... per rimuovere righe quella colonna vuote...
df_drop = df_drop.dropna(subset=["sepal_width"])

#rimozione per colonna, possibile indicare soglia minima...
df_drop = df.copy()
df_drop = df_drop.dropna(axis=1, thresh=145)
print(df_drop.shape)

# - per calcolare percentuale al 90% del numero osservazioni:
thresh = int(df.shape[0]*0.9)

*Imputazione dei dati feature mancanti, cioè sostituire NA (media, mediana, moda)*

*  pandas (su singola colonna)

In [ ]:
df = pd.read_csv(BASE_URL+"iris_missing.csv", index_col=0)
df.head()

df_imp = df.copy()

col = "sepal_width"
replace_with = round(df_imp[col].mean(), 1)
# - si poteva fare anche con la median
# - si poteva fare anche con la moda, prendendo il più frequente
#   replace_with = round(df_imp[col].mode()[0], 1)

df_imp[col] = df_imp[col].fillna(replace_with)
df_imp.head()
print(df_imp.isna().sum()) # ora gli NA sono zero per la colonna

*  pandas (su più colonne)

In [ ]:
df = pd.read_csv(BASE_URL+"iris_missing.csv", index_col=0)
df.head()

df_imp = df.copy()

replace_with = round(df_imp.mean(numeric_only=True), 1) #series con valori medi per colonna
df_imp = df_imp.fillna(replace_with)
df_imp.head()
print(df_imp.isna().sum()) # tutti gli NA sono zero

*  numpy e scikit-learn

In [ ]:
from sklearn.impute import SimpleImputer

df = pd.read_csv(BASE_URL+"iris_missing.csv", index_col=0)

X = df.drop("species", axis=1).values #si rimuove variabile target, che non va adeguata..
X.shape

imp = SimpleImputer(strategy="mean")
X_imp = imp.fit_transform(X)
X_imp[:5]
np.isnan(X).sum(axis=0)
np.isnan(X_imp).sum(axis=0) #valori NaN per asse colonna

*Normalizzazione*

x-min(x) / max(x) - min(x)

*  pandas

In [ ]:
df = pd.read_csv(BASE_URL+"wine.csv", usecols=[0,1,7]) #usiamo solo alcune featues per l'esercizio
df.head()

df_norm = df.copy()
features = ["alcol", "flavonoidi"] #escludiamo il target che è classe, che non ha senso normalizzare
to_norm = df_norm[features]
df_norm[features] = (to_norm-to_norm.min()) / (to_norm.max()-to_norm.min())
df_norm.head() #così abbiamo tutto tra min 0 e max 1

*  numpy e scikit-learn

In [ ]:
from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv(BASE_URL+"wine.csv", usecols=[0,1,7]) #usiamo solo alcune featues per l'esercizio
X = df.drop("classe", axis=1).values #rimuoviamo colonna target
X.shape

mms = MinMaxScaler()
X_norm = mms.fit_transform(X)
X_norm[:5] #così abbiamo tutto tra min 0 e max 1

*Standardizzazione*

x-mean(x) / sd(x)

*  pandas

In [ ]:
df = pd.read_csv(BASE_URL+"wine.csv", usecols=[0,1,7]) #usiamo solo alcune featues per l'esercizio
df.head()

df_std = df.copy()
features = ["alcol", "flavonoidi"] #escludiamo il target che è classe, che non ha senso standardizzare
to_std = df_std[features]

df_std[features] = (to_std - to_std.mean()) / to_std.std(ddof=0) #ddof per non fare -1 (std campionaria) ma della popolazione (come numpy)
df_std.head() #infatti la media è quasi uguale a zero e la deviazione standard prossima a 1

*  scikit-learn

In [ ]:
from sklearn.preprocessing import StandardScaler

df = pd.read_csv(BASE_URL+"wine.csv", usecols=[0,1,7]) #usiamo solo alcune featues per l'esercizio
X = df.drop("classe", axis=1).values #rimuoviamo colonna target
X.shape

ss = StandardScaler()
X_std = ss.fit_transform(X)
X_std[:5]
print(X_std.mean()) # prossimo zero
print(X_std.std()) # prossimo a 1

## Dataset + Data Preprocessing (esercizio)

In [ ]:
import pandas as pd

DATASET_URL = "https://raw.githubusercontent.com/ProfAI/machine-learning-fondamenti/main/datasets/housing_dirty.csv"

df = pd.read_csv(DATASET_URL, index_col=0)
df.head()

### numero righe/colonne dataset
df.shape

### tipologia variabile e distribuzione
df.info()
# per quantitative...
df.describe()
# per qualitative numero occorrenze...
df["CRIM"].value_counts()
df["CHAS"].value_counts()

### valori mancanti per colonna
df.count()
df.isna().sum()

### rimuovi le colonne con il 30% di dati mancanti
df.count()<df.shape[0]*0.7 #df = df.drop("LSTAT", axis=1)
thresh = df.shape[0]*0.7
df = df.dropna(thresh=thresh, axis=1)
df.head()

### rimuovi le righe con il 75% di dati mancanti
df = df.dropna(thresh=df.shape[1]*0.75)
df.shape

### rimuovi righe con campo mancante
df = df.dropna(subset=["PRICE"])
df["PRICE"].isna().sum()
df.shape

### imputazione per valore medio su variabili quantitative
cols = df.columns
cols = cols.drop(["CRIM", "CHAS"])
replace_with = df[cols].mean()
df = df.fillna(replace_with)
df.isna().sum()

### codifica di variabili qualitative
mapping = {"LOW":1, "MODERATE":2, "HIGH":3, "VERY HIGH":4}
df["CRIM"] = df["CRIM"].map(lambda t: mapping[t])
mapping = {"YES":1, "NO":0}
df["CHAS"] = df["CHAS"].map(lambda t: mapping[t])
df.head()

### imputazione con moda su variabili qualitative
cols = ["CRIM", "CHAS"]
replace_with = df[cols].mode()
df[cols] = df[cols].fillna(replace_with)
df.head()

### standardizzazione
features = df.columns.drop("PRICE")
df[features] = (df[features] - df[features].mean())  / df[features].std()
df[features].mean()
df[features].std()

### salva dataframe in tsv
df.to_csv("housing_cleaned.tsv", sep="\t")

## Regressione lineare

*Regressione lineare semplice*

*  numpy

In [ ]:
import numpy as np

In [ ]:
class LinearRegression:
  coef = None #peso
  intercept = None #bias

  def fit(self, x, y):
    x_sum = x.sum()
    y_sum = y.sum()
    xy_sum = (x*y).sum()
    x2_sum = (x*x).sum()
    n = y.shape[0]

    self.coef = (n*xy_sum - x_sum*y_sum) / (n*x2_sum - x_sum*x_sum)
    self.intercept = (y_sum-self.coef * x_sum) / n

  def predict(self, x):
    return self.coef*x + self.intercept

In [ ]:
x_train = np.array([80,150,30,50,120,60,110,110]) #metri quadri
y_train = np.array([16,30,12,10,24,18,20,25]) #valore casa x10000$

lr = LinearRegression()
lr.fit(x_train, y_train)

print(lr.coef)
print(lr.intercept)

In [ ]:
lr.predict(70) # esempio di predizione
y_pred = lr.predict(x_train)

*  Funzioni di costo (valore minore  = maggiore bontà del modello)
*  Funzioni di scoring (valore maggiore  = maggiore bontà del modello)

In [ ]:
#[COST] errore assoluto medio (MAE), indica di quanto più o meno sbaglia il modello
def mean_absolute_error(y_true, y_pred):
  return np.abs(y_true - y_pred).sum() / y_true.shape[0]

#[COST] errore quadratico medio (MSE)
def mean_squared_error(y_true, y_pred):
    return np.square(y_true - y_pred).sum() / y_true.shape[0]

#[COST] radice dell'errore quadratico medio (RMSE)
def root_mean_squared_error(y_true, y_pred):
  return np.sqrt(mean_squared_error(y_true, y_pred))

#[SCORE] R sqared R2
def rss(y_true, y_pred): #Somma quadrati residui
  return np.power(y_true-y_pred, 2).sum() #MSE senza divisione per R
def sst(y_true): #Somma quadrati totali
  return np.power(y_true-y_true.mean(), 2).sum()
def r2_score(y_true, y_pred):
  return 1 - rss(y_true, y_pred) / sst(y_true)

In [ ]:
mean_absolute_error(y_train, y_pred) #stessa scala dei dati

In [ ]:
#entrambi sempre maggiori del MSE
mean_squared_error(y_train, y_pred) #scala diversa dai dati, da riportare con radice quadrata
root_mean_squared_error(y_train, y_pred) #qui sono penalizzati errori più grandi del modello

In [ ]:
# 0.3 - 0.5 = scarso
# 0.5 - 0.7 = discreto
# 0.7 - 0.9 = buono
# 0.9 - 1.0 = ottimo
r2_score(y_train, y_pred)

*  Rappresentazione grafica della predizione rispetto ai valori reali

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figsize=(10,8)

x_line = np.arange(1, 150) # valori da 0 a 150
y_line = lr.predict(x_line)

plt.grid()
plt.scatter(x_train, y_train, c="green")
plt.plot(x_line, y_line, c="red")
plt.show()

*  scikit-learn

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_errore, mean_absolute_errore, r2_score

x_train = np.array([[80],[150],[30],[50],[120],[60],[110],[110]]) #metri quadri (anche per una feature, bidimensionale in scikit-learn)
y_train = np.array([16,30,12,10,24,18,20,25]) #valore casa x10000$

lr = LinearRegression()
lr.fit(x_train, y_train)
lr.coef_
lr.intercept_

y_pred = lr.predict(x_train)

mean_absolute_error(y_train, y_pred) #MAE
mean_squared_error(y_train, y_pred) #MSE
sqrt(mean_squared_error(y_train, y_pred)) #RMSE
r2_score(y_train, y_pred) #R2

*Regressione lineare multipla*

Con più variabili il modello diventa più accurato

In [ ]:
x_train = np.array([[80,1995],[150,1995],[30,2008],[50,1996],[120,1994],[60,2006],[110,1989],[110,2000]]) #metri quadri e anno costruzione
y_train = np.array([16,30,12,10,24,18,20,25]) #valore casa x10000$

lr = LinearRegression()
lr.fit(x_train, y_train)

y_pred = lr.predict(x_train)

mean_absolute_error(y_train, y_pred) #MAE
mean_squared_error(y_train, y_pred) #MSE
sqrt(mean_squared_error(y_train, y_pred)) #RMSE
r2_score(y_train, y_pred) #R2

*Matrice di correlazione*

Se abbiamo tante features non possiamo usarle tutte, ma dobbiamo prendere quella più significative.

Se due features hanno alta correlazione una è superflua in quanto si determinano a vicenda

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

In [ ]:
data = [[80,1995,16],[150,1995,30],[30,2008,12],[50,1996,10],[120,1994,24],[60,2006,18],[110,1989,20],[110,2000,25]] #metri quadri, anno costruzione, target(prezzo)

df = pd.DataFrame(data, columns=["Dimensione", "Anno", "Valore"])
df.corr() #indice correlazione di Pearson di default
sb.heatmap(df.corr(), annot=True) #rappresentazione visiva della correlazione

# se avevamo un array numpy e non un dataset avremmo potuto passare i nomi delle colonne
data = df.corr().values
columns = ["Dimensione", "Anno", "Valore"]
sb.heatmap(data, annot=True, xticklabels=columns, yticklabels=columns)

*Regressione lineare polinominale*

In questo caso dopo il grado 10 la complessità aumenta e non è più efficace.

Con troppi gradi però rischia di essere però troppo legato al dataset di partenza.

In [ ]:
x_train = np.array([[80],[150],[30],[50],[120],[60],[110],[110]]) #metri quadri (anche per una feature, bidimensionale in scikit-learn)
y_train = np.array([16,30,12,10,24,18,20,25]) #valore casa x10000$

from sklearn.preprocessing import PolynomialFeatures
np.set_printoptions(suppress=True)

poly = PolynomialFeatures(2, include_bias=True) #2 gradi
poly.fit(x_train)

x_train_poly = poly.transform(x_train)
x_train_poly.shape #1° colonna sempre il bias uguale a 1, poi la x e la x^2

#verifica del modello su più gradi
degrees = [2, 3, 4, 5, 10, 20, 50, 100]

x_line = np.arange(1,150) #x su cui facciamo la previsione

for d in degrees:
  poly = PolynomialFeatures(d)
  x_train_poly = poly.fit_transform(x_train)

  lr = LinearRegression()
  lr.fit(x_train_poly, y_train)

  y_pred = lr.predict(x_train_poly)
  mse = mean_squared_error(y_train, y_pred)
  r2 = r2_score(y_train, y_pred)

  #su x line dobbiamo creare feature polinomiali, i dati da prevedere devono avere stesso trattamento di quelli dell'addestramento
  x_line_poly = poly.transform(x_line.reshape(-1,1)) #deve essere bidimensionale per scikit-learn (reshape)
  y_line = lr.predict(x_line_poly)

  plt.figure(figsize=(6,4))
  plt.grid()

  plt.scatter(x_train, y_train, c="green")
  plt.plot(x_line, y_line, c="red")

  plt.ylim(0,30)
  plt.text(0, 28, f"Degree: {d}")
  plt.text(125, 5, f"MSE: {mse:.2f}")
  plt.text(125, 2, f"R2: {r2:.2f}")

  plt.show()

*Regressione lineare polinominale multidimensionale*

In [ ]:
x_train = np.array([[80,1995],[150,1995],[30,2008],[5,19960],[120,1994],[60,2006],[110,1989],[110,2000]]) #metri quadri, anno
y_train = np.array([16,30,12,10,24,18,20,25]) #valore casa x10000$

poly = PolynomialFeatures()

x_train_poly = poly.fit_transform(x_train)
x_train_poly.shape

lr = LinearRegression()
lr.fit(x_train_poly, y_train)

y_pred = lr.predict(x_train_poly)

#valori molto alti, probabilmente dovuti a overfitting!
mean_squared_error(y_train, y_pred)
r2_score(y_train, y_pred)

## Regressione lineare (esercizio)

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

DATASET_URL = "https://raw.githubusercontent.com/ProfAI/machine-learning-fondamenti/main/datasets/housing.csv"
BASE_URL = "https://raw.githubusercontent.com/ProfAI/machine-learning-fondamenti/main/datasets/"

df = pd.read_csv(DATASET_URL, index_col=0)
df.head()

### 1) matrice correlazione
plt.figure(figsize=(10,8))
sns.heatmap(df.corr(), annot=True)
#spiccano ad esempio LSTAT (tasso povertà) e RM (numero stanze) relativi al valore predetto PRICE
#si notano poi relazioni forti anche tra alcune features

### 2) modello regressione lineare sulla principale feature (tasso povertà)
def evaluate(model, dataset):
  x, y = dataset
  y_pred = model.predict(x)
  print(f"MSE: {mean_squared_error(y, y_pred):.2f}")
  print(f"R2 {r2_score(y, y_pred):.2f}")

x = df[["LSTAT"]].values #doppia parentesi, deve essere bidimensionale per scikit-learn
y = df["PRICE"].values

lr = LinearRegression()
lr.fit(x,y)

#MSE alto da raffrontare coi valori di y
evaluate(lr, (x,y))
y.min()
y.max()
y.mean()
#il modello non è buono!

### 3) modello regressione lineare multipla con 2 variabili più significative
#si provano LSTAT e RM, che però sono molto collegate anche tra loro, quindi al massimo si passa alla successiva
x = df[["LSTAT", "RM"]].values
lr = LinearRegression()
lr.fit(x,y)
evaluate(lr, (x,y))
#il modello migliora ma di poco...

x = df[["LSTAT", "PTRATIO"]].values
lr = LinearRegression()
lr.fit(x,y)
evaluate(lr, (x,y))
#il modello non migliora neanche così

### 4) modello con terza variabile, provando polinomiale senza superare grado 5 con o senza bias
x = df[["LSTAT", "RM", "PTRATIO"]].values

for i in range(1,6):
  poly = PolynomialFeatures(i, include_bias=False) #si prova con o senza bias
  x_poly = poly.fit_transform(x)

  lr = LinearRegression()
  lr.fit(x_poly,y)

  print(f"Polinomio di grado: {i} con bias")
  evaluate(lr, (x_poly, y))
  print(f"--------------------------------")

### 5) modello regressione lineare con tutte le variabili
x = df.drop("PRICE", axis=1).values #togliamo quindi solo il target
y = df["PRICE"].values

lr = LinearRegression()
lr.fit(x,y)
evaluate(lr, (x,y))
#modello buono 0.7

### 6/7) normalizzazione/standardizzazione, cambia qualcosa?
mms = MinMaxScaler()
x_norm = mms.fit_transform(x)

lr = LinearRegression()
lr.fit(x_norm,y)
evaluate(lr, (x_norm,y))
#--------------------------
sc = StandardScaler()
x_std = sc.fit_transform(x)

lr = LinearRegression()
lr.fit(x_std,y)
evaluate(lr, (x_std,y))
#non cambia nulla sul metodo dei minimi quadrati
#hanno senso quando c'è un processo di ottimizzazione iterativo

### 8/9) utilizzo modello su un file csv e salvataggio risultato
x = df.drop("PRICE", axis=1).values
y = df["PRICE"].values

lr = LinearRegression()
lr.fit(x,y)

df_pred = pd.read_csv(BASE_URL + "housing_predict.csv")
df_pred.head()

#droppiamo owner che non ha importanza, è un codice identificativo, non ripetuto senza informazioni utili
x_pred = df_pred.drop("OWNER", axis=1).values
y_pred = lr.predict(x_pred)

df_result = pd.DataFrame(
    {
      "owner":df_pred["OWNER"].values,
      "estimated price":y_pred,
    }
)
df_result.to_excel("housing_estimate.xlsx", index=False) #salviamo rimuovendo colonna di indice

## Overfitting

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.datasets import make_regression

RANDOM_SEED = 0

#generazione di un dataset affetto da ovefitting, ad esempio con n variabili = n osservazioni
x, y = make_regression(
    n_samples = 100,
    n_features = 100,
    n_informative = 10,
    n_targets = 1,
    random_state = RANDOM_SEED
)

*Hold out*

Mettiamo una parte di dati da parte per il test

In [ ]:
from sklearn.model_selection import train_test_split

# impostiamo un set di test del 30%, se ho molti dati posso ridurlo..
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=RANDOM_SEED)
x_train.shape #70
x_test.shape #30

#standardizziamo i dati
ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.transform(x_test)

lr = LinearRegression()
lr.fit(x_train, y_train)

def evaluate_model(model, dataset):
  x, y = dataset
  y_pred = model.predict(x)
  print(f"MSE: {mean_squared_error(y, y_pred):.3f}")
  print(f"R2 {r2_score(y, y_pred):.3f}")

evaluate_model(lr, (x_train, y_train)) # r2 = 1, perfetto, da vedere però sul test set...
evaluate_model(lr, (x_test, y_test)) # disastro, r2 = 0.4

*Cross validation*

In [ ]:
from sklearn.model_selection import cross_val_score

lr = LinearRegression()
score = cross_val_score(lr, x, y, cv=5, scoring="r2") #numero batch = 5
score # array con r2 per ognuno dei 5 modelli
score.mean() #0.69

from sklearn.model_selection import cross_validate

lr = LinearRegression()
cv_result = cross_validate(lr, x, y, cv=5, return_train_score=True) # risultato anche addrestamento oltre che test
cv_result["train_score"].mean() # media di r2 a 1, caso overfitting
cv_result["test_score"].mean() # qui otteniamo 0.69, confermiamo overfitting



Sopra non abbiamo fatto standardizzazione, perchè andrebbe fatto dentro i metodi di scikit-learn, dobbiamo riscriverci la funzione

In [ ]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np

kf = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
kf.get_n_splits(x) # 5 batch

train_score = []
test_score = []

for train_index, test_index in kf.split(x):
  x_train, x_test = x[train_index], x[test_index]
  y_train, y_test = y[train_index], y[test_index]

  #ora possiamo standardizzare...
  ss = StandardScaler()
  x_train = ss.fit_transform(x_train)
  x_test = ss.transform(x_test)

  lr = LinearRegression()
  lr.fit(x_train, y_train)

  r2_train = r2_score(y_train, lr.predict(x_train))
  r2_test = r2_score(y_test, lr.predict(x_test))

  train_score.append(r2_train)
  test_score.append(r2_test)

scores = {
    "train_score": np.array(train_score),
    "test_score": np.array(test_score)
}
#anche qui stesso risultato, con train_score sempre a 1, che suggerisce overfitting e test_score piuttosto basso

*LOOCV*

Leave one out cross validation

Addestramento su tutto il dataset a eccezione di un unico esempio, utili se abbiamo un dataset con pochi esempi a disposizione.

In pratica è come un KFold con numero batch = numero esempi

In [ ]:
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np

loo = LeaveOneOut()
loo.get_n_splits(x) # 100 batch

train_cost = []
test_cost = []

#r2 non si può usare col test su un solo esempio, quindi useremo mse
for train_index, test_index in loo.split(x):
  x_train, x_test = x[train_index], x[test_index]
  y_train, y_test = y[train_index], y[test_index]

  #ora possiamo standardizzare...
  ss = StandardScaler()
  x_train = ss.fit_transform(x_train)
  x_test = ss.transform(x_test)

  lr = LinearRegression()
  lr.fit(x_train, y_train)

  mse_train = mean_squared_error(y_train, lr.predict(x_train))
  mse_test = mean_squared_error(y_test, lr.predict(x_test))

  train_cost.append(mse_train)
  test_cost.append(mse_test)

costs = {
    "train_cost": np.array(train_cost),
    "test_cost": np.array(test_cost)
}

costs["train_cost"].mean() # praticamente zero, indica che è perfetto
costs["test_cost"].mean() # valore altissimo, altra conferma di overfitting

*Modelli di correzione overfitting*

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=RANDOM_SEED)

#standardizziamo i dati
ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.transform(x_test)

#modello lineare sempice minimi quadrati
lr = LinearRegression()
lr.fit(x_train, y_train)

evaluate_model(lr, (x_train, y_train)) # overfitting, quindi r2 a 1
evaluate_model(lr, (x_test, y_test)) # overfitting, quindi r2 sul test molto basso

*  Ridge regression (regolarizzazione di tipo 2, meno invasiva)  - L2

In [ ]:
from sklearn.linear_model import Ridge
model = Ridge(alpha=1.) #alfa è il labda
model.fit(x_train, y_train)
evaluate_model(model, (x_train, y_train)) # ancora altissimo r2
evaluate_model(model, (x_test, y_test)) # peggiorato rispetto a prima
#caso di underfitting, alzando alpha peggiora ancora

*  LASSO regression (regolarizzazion di tipo 1) - L1

Questa permette di portare il peso di feature ritenute non significative a zero

In [ ]:
from sklearn.linear_model import Lasso
model = Lasso(alpha=1.) #alfa è il labda
model.fit(x_train, y_train)
evaluate_model(model, (x_train, y_train)) # ancora altissimo r2
evaluate_model(model, (x_test, y_test)) # ora r2 rimane molto alto a 0.99, quindi la normalizzazione L1 ha rimosso overfitting
#sono state elminate su 100, 90 feature ritenute no valide

*  Elastic NET

Permette di combinare entrambe (L1+L2)

In [ ]:
from sklearn.linear_model import ElasticNet

model = ElasticNet(alpha=1., l1_ratio=1) #alfa è il labda
model.fit(x_train, y_train)
evaluate_model(model, (x_train, y_train))
evaluate_model(model, (x_test, y_test)) # uguale al precente perchè abbiamo messo l1_ratio a 100%

*  Learing Curve

Facciamo curva sul modello migliore identificato, ovvero Lasso.

In [ ]:
from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()

train_sizes_abs, train_scores, test_scores = learning_curve(
    Lasso(),
    x,
    y,
    random_state = RANDOM_SEED
)

plt.plot(train_sizes_abs, train_scores.mean(axis=1), label="Train score")
plt.plot(train_sizes_abs, test_scores.mean(axis=1), label="Test score")
plt.show()

#Si parte da soluzione di overfitting, poi aumentando gli esempi si raggiunge un valore perfetto


## Regolarizzazione

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import LeaveOneOut, KFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import Ridge,Lasso ,LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score

RANDOM_SEED = 0

DATASET_URL = "https://raw.githubusercontent.com/ProfAI/machine-learning-fondamenti/main/datasets/housing.csv"

df = pd.read_csv(DATASET_URL, index_col=0)
df.head()

x = df.drop("PRICE", axis=1).values # array con feature
y = df["PRICE"].values # array con target

#creazione array per addetramneto e test
x_train, x_test, y_train , y_test = train_test_split(x, y, test_size=.3, random_state=RANDOM_SEED)
x_train.shape #70% del totale

#creazione feature polinomiali di seconod grado
poly = PolynomialFeatures(degree=2)
x_train = poly.fit_transform(x_train)
x_test = poly.transform(x_test)

#standardizzazione
ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.transform(x_test)

def evaluate_model(model, dataset):
  x, y = dataset
  y_pred = model.predict(x)
  print(f"MSE: {mean_squared_error(y, y_pred):.3f}")
  print(f"R2 {r2_score(y, y_pred):.3f}")

#iniziamo a creare modelli
lr = LinearRegression()
lr.fit(x_train, y_train)
evaluate_model(lr, (x_train, y_train)) #0.95
evaluate_model(lr, (x_test, y_test)) #0.63

1 - (0.639/0.95) #32% di differenza, l'esercizio chiede sotto al 15...proviamo a ottimizzare

model = Ridge(alpha=1.)
model.fit(x_train, y_train)
evaluate_model(model, (x_train, y_train))
evaluate_model(model, (x_test, y_test)) #situazione migliorata, provamiamo regolarizzazione L1 (Lasso)

model = Lasso(alpha=1.)
model.fit(x_train, y_train)
evaluate_model(model, (x_train, y_train))
evaluate_model(model, (x_test, y_test)) #peggio

# ora usiamo la cross-validation
train_score = []
test_score = []

kf = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)

for train_index, test_index in kf.split(x):
  x_train, x_test = x[train_index], x[test_index]
  y_train, y_test = y[train_index], y[test_index]

  poly = PolynomialFeatures(degree=2)
  x_train = poly.fit_transform(x_train)
  x_test = poly.transform(x_test)

  ss = StandardScaler()
  x_train = ss.fit_transform(x_train)
  x_test = ss.transform(x_test)

  model = Ridge(alpha=10.)
  model.fit(x_train, y_train)

  r2_train = r2_score(y_train, model.predict(x_train))
  r2_test = r2_score(y_test, model.predict(x_test))

  train_score.append(r2_train)
  test_score.append(r2_test)

scores = {
    "train_score": np.array(train_score),
    "test_score": np.array(test_score),
}

scores["train_score"].mean()
scores["test_score"].mean()
1 - (scores["test_score"].mean()/scores["train_score"].mean()) #5% di differenza, ottimo risultato

#creiamo anche laerning curve del modello migliore
sns.set_theme()

train_sizes_abs, train_scores, test_scores = learning_curve(
    Ridge(alpha=10.),
    x,
    y,
    random_state = RANDOM_SEED
)

plt.plot(train_sizes_abs, train_scores.mean(axis=1), label="Train score")
plt.plot(train_sizes_abs, test_scores.mean(axis=1), label="Test score")
plt.show()

## Classificazione

*Classificazione binaria*

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression

In [ ]:
RANDOM_SEED = 0

In [ ]:
x, y = make_classification(n_samples = 100,
                           n_features = 2,
                           n_informative = 2,
                           n_redundant = 0,
                           n_repeated = 0,
                           n_classes = 2, #binaria
                           random_state = RANDOM_SEED)

x[:,0] #prima feature
x[:,1] #seconda feature

plt.scatter(x[:,0], x[:,1], c=y) #colora pallini con le categorie
#lo scopo è trovare retta che taglia i due gruppi

#creaiamo il modello
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.3, random_state=RANDOM_SEED)

lr = LogisticRegression()
lr.fit(x_train, y_train)

def plot_decision_boundary(model, dataset):
  x,y = dataset
  h = .02

  x_min, x_max = x[:,0].min() -.2, x[:,0].max() + .2
  y_min, y_max = x[:,1].min() -.2, x[:,1].max() + .2

  xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                       np.arange(y_min, y_max, h))

  z = model.predict(np.c_[xx.ravel(), yy.ravel() ]) # ravel array multi a single dimension
  z = z.reshape(xx.shape)

  plt.contourf(xx, yy, z, cmap=plt.cm.Paired)
  plt.scatter(x[:,0], x[:,1], c=y, edgecolor='white')

plot_decision_boundary(lr, (x_train, y_train)) # quello che divide meglio le classi
plot_decision_boundary(lr, (x_test, y_test)) # anche su quello di test



*Valutazione del modello*

*  log loss

basata sulla probabilità della regressione logistica, quindi no usiamo predict ma predict_proba

In [ ]:
from sklearn.metrics import log_loss

# tra 0 e 1, probabilità appartenza classe 1 o 2
y_proba_train = lr.predict_proba(x_train)
y_proba_test = lr.predict_proba(x_test)

#log loss uguale a zero indica modello perfetto
log_loss(y_train, y_proba_train)
log_loss(y_test, y_proba_test)

*  matrice di confusione

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pandas as pd

y_pred_train = lr.predict(x_train)
y_pred_test = lr.predict(x_test)

cm = confusion_matrix(y_train, y_pred_train)
#matrice TN PF
#        FN TP

#mostriamo graficamente...
def plot_confusion_matrix(y_true, y_pred):
  cm = confusion_matrix(y_true, y_pred)
  df_cm = pd.DataFrame(cm,
                        index=["Negative","Positive"],
                        columns=["Predicted Negative", "Predicted Positive"])
  sns.heatmap(df_cm, annot=True)

plot_confusion_matrix(y_train, y_pred_train)
plot_confusion_matrix(y_test, y_pred_test)

*  Accuracy, Precision, Recall, F1

score, valori più alti = migliore qualità modello

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def classification_report(y_true, y_pred):
  print(f"PRECISION: {precision_score(y_true, y_pred)}") #%classificazioni positive effettivamente tali
  print(f"RECALL: {recall_score(y_true, y_pred)}")#%classificazioni positive effettivamente classificate come positive
  print(f"F1: {f1_score(y_true, y_pred)}") #media aarmonica tra precision e recall
  print(f"ACCURACY: {accuracy_score(y_true, y_pred)}") #% modello ha categorizzato corretamente

classification_report(y_train, y_pred_train)
classification_report(y_test, y_pred_test)

#in alternativa...
from sklearn.metrics import classification_report
print(classification_report(y_train, y_pred_train))
print(classification_report(y_test, y_pred_test))

*  ROC curve

In [ ]:
from sklearn.metrics import RocCurveDisplay, roc_auc_score
RocCurveDisplay.from_estimator(lr, x_train, y_train) #buon modello, valore alto
RocCurveDisplay.from_estimator(lr, x_test, y_test)

*  Classificaizone multi classe

In [ ]:
y_proba_train = lr.predict_proba(x_train)
roc_auc_score(y_train, y_proba_train, multi_class="ovo", average="macro") #oppure ovr o multinomial

## Classificazione (esercizio)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import RocCurveDisplay, roc_auc_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.preprocessing import StandardScaler

RANDOM_SEED = 0

### import del dataset
DATASET_URL = "https://raw.githubusercontent.com/ProfAI/machine-learning-fondamenti/main/datasets/breast_cancer.csv"
df = pd.read_csv(DATASET_URL)
df.head()
df.count() #563 righe e 32 colonne, con colonna

#ID univoco, che è inutile ai fini del modello...
df = df.drop("ID number", axis=1)
df.head()

#dataset sbilanciato verso tumori benigni
counts = df["diagnosis"].value_counts()
print(f"Tumori benigni: {counts[0]} / ({counts[0]/counts.sum()*100:.2f}%)")

### codifichiamo il target in numero
map_dict = {"M":1, "B":0}
df["diagnosis"] = df["diagnosis"].map(lambda x: map_dict[x])
df.head()

### creazione del modello...
x = df.drop("diagnosis", axis=1) #array features
y = df["diagnosis"].values # array target
#dividiamo in train e test set
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.3, random_state=RANDOM_SEED)
x_train.shape #70%
x_test.shape #70%

#standardizzazione..
ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.transform(x_test)

#visto che il dataset è sbilanciato, assegniamo peso in base alla proporzione
lr = LogisticRegression(class_weight="balanced")
lr.fit(x_train, y_train)

## valutazione del modello
y_pred_train = lr.predict(x_train)
y_proba_train = lr.predict_proba(x_train)
y_pred_test = lr.predict(x_test)
y_proba_test = lr.predict_proba(x_test)

print(classification_report(y_train, y_pred_train))
print(classification_report(y_test, y_pred_test)) #accuracy 0.98, recall invece da requisiti deve essere 1 (nessun falso positivo)
# in questo caso potremmo spostare il threshold visto che i positivi hanno più peso dei negativi...
y_pred_train = np.where(y_proba_train[:,1]>0.25, 1, 0) #di default è a 0.5
y_pred_test = np.where(y_proba_test[:,1]>0.25, 1, 0) #di default è a 0.5
print(classification_report(y_train, y_pred_train))
print(classification_report(y_test, y_pred_test)) # ora è ok, recall di uno sul dataset di test

## grafico confusion matrix
def plot_confusion_matrix(y_true, y_pred, labels=["Negative", "Positive"], show_precision_recall=False):
  cm = confusion_matrix(y_true, y_pred) # tn, fp, fn, tp
  df_cm = pd.DataFrame(cm, index=labels,
                       columns=["Predicated " + labels[0], "Predicted " + labels[1]])
  sns.heatmap(df_cm, annot=True, fmt="g")
  if show_precision_recall:
    plt.text(0,-0.1, f"Precision: {cm[1][1]/cm[1][1]+cm[0][1]}") #-0.1 è per miglire visualizzazione
    plt.text(0,-0.1, f"Recall: {cm[1][1]/(cm[1][1]+cm[1][0])}") #-0.1 è per miglire visualizzazione

plot_confusion_matrix(y_train, y_pred_train, show_precision_recall=True)
plot_confusion_matrix(y_test, y_pred_test, show_precision_recall=True) #nessun falso positivo come da requisito

## roc curve
RocCurveDisplay.from_estimator(lr, x_train, y_train, name="Cancer Classifier")

## classificazioni sui nuovi dati e salvataggio in file excel
DATASET_URL = "https://raw.githubusercontent.com/ProfAI/machine-learning-fondamenti/main/datasets/breast_cancer_pred.csv"
df_pred = pd.read_csv(DATASET_URL)
x = df.drop("ID number", axis=1).values
x = ss.transform(x)

y_proba = lr.predict_proba(x)
y_pred = np.where(y_proba[:,1]>0.25, 1, 0)
y_pred

df_results = pd.DataFrame({
    "ID number": df_pred["ID number"],
    "prediction": y_pred,
    "probability": y_proba.max(axis=1).round(4) # la classe con più probabilità
})

df_results.head()
df_results.to_excel("breast_cancer_prediction.xlsx")

## Clustering

*K-means*

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.rcParams["figure.figsize"] = (16,10)
sns.set_theme()

In [ ]:
RANDOM_SEED = 2

from sklearn.datasets import make_blobs
x, _ = make_blobs(n_samples = 100,
                  n_features = 2,
                  centers = 3,
                  cluster_std = 0.5,
                  random_state= RANDOM_SEED)

# es. 1° feature spesa in birra dei clienti di un market
# es. 2° feature spesa in pannolini dei clienti di un market

#centriamolo a zero e aumentiamo i valori per avere dei dati di test più sensati
x-=x.min(axis=0)
x[:,0]*=20
x[:,1]*=6

In [ ]:
sns.scatterplot(x=x[:,0],y=x[:,1], s=100) #risultano 3 cluster

In [ ]:
#creazione del modello
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=RANDOM_SEED) # i centroidi all'inizio sono a caso, non ottimale, potrebbero essere troppo vicini
kmeans = KMeans(n_clusters=3, init="k-means++") # in questo modo sono maggiornamente tarati
kmeans.fit(x)

In [ ]:
#valutazione del modello (inertia o distotion)
kmeans.cluster_centers_ #array con i centroidi

from scipy.spatial.distance import cdist

#esempio utilizzo...
a = np.array([[1,2],[3,2]])
b = np.array([[2,1],[3,3]])
cdist(a, b, "euclidean") # array numpi con distanze tra i punti

distortion = sum(np.square(np.min(cdist(x, kmeans.cluster_centers_, 'euclidean'), axis=1)) / x.shape[0]) #107
inertia = sum(np.square(np.min(cdist(x, kmeans.cluster_centers_, 'euclidean'), axis=1))) #uguale, senza media
#...oppure:
kmeans.inertia_

In [ ]:
#visualizzazione del cluster
y_kmeans = kmeans.predict(x)
y_kmeans

centers = kmeans.cluster_centers_

#aggiungiamo informazioni dedotte dal grafico
cluster_map = {0: "Neo papà", 1:"Donne single", 2:"Neo mamme"}
vfunc = np.vectorize(lambda x: cluster_map[x])
classes = vfunc(y_kmeans)

plt.xlabel("Spesa in birra")
plt.xlabel("Spesa in pannolini")

#mostriamo colori rappresentazioni ottenute, con indicazione dei centroidi
sns.scatterplot(x=x[:,0], y=x[:,1], hue=classes, s=100)
plt.scatter(centers[:,0], centers[:,1], c="red", s=200, alpha=0.5)



In [ ]:
#elbow method, per n valori di k addestrati

ssd = {}

for k in range(1,10):
  kmeans = KMeans(n_clusters=k, init="k-means++").fit(x)
  ssd[k] = kmeans.inertia_

ssd

plt.plot(list(ssd.keys()), list(ssd.values()), marker="o")
plt.xlabel('Number of clusters (k)')
plt.ylabel('Sum of squared distances (Inertia)')
plt.title('Elbow Method For Optimal k')
plt.show()

#dal gafico si conferma che il "gomito" è a 3

## Clustering (esercizio)

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
import seaborn as sns
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans

RANDOM_SEED = 1

plt.rcParams["figure.figsize"] = (16,10)
sns.set_theme()

DATASET_URL = "https://raw.githubusercontent.com/ProfAI/machine-learning-fondamenti/main/datasets/mall_customers.csv"
df = pd.read_csv(DATASET_URL)
df.head()

#funzione per stampare la curva SSD per diversi k
def plot_ssd_curve(x, k_range=(1,10)):
  ssd = {}
  for k in range(k_range[0], k_range[1]):
    kmeans = KMeans(n_clusters = k, init="k-means++", random_state=RANDOM_SEED)
    kmeans.fit(x)
    ssd[k] = kmeans.inertia_

  plt.plot(list(ssd.keys()), list(ssd.values()), marker="o")

  plt.xlabel("Numero di cluster", fontsize=16)
  plt.ylabel("Somma della distanza al quadrato", fontsize=16)

  plt.show()

def plot_clusters(model, data, axLabels=None, print_ssd=False):
  centers = model.cluster_centers_
  y_kmeans = model.predict(data)
  sns.scatterplot(x=data[:,0], y=data[:,1], hue=y_kmeans, s=100)
  plt.scatter(centers[:,0], centers[:,1], c="red", alpha=.5)

  if axLabels!=None:
    plt.xlabel(axLabels[0], fontsize=16)
    plt.ylabel(axLabels[1], fontsize=16)
  if print_ssd:
    plt.text(x[:,0].max()-10, 0, f"SSD={model.inertia_:.2f}")
  plt.show()

#modello su age e spending score...
x = df[["Age", "Spending Score (1-100)"]].values
plot_ssd_curve(x) #si vede k=4

kmeans = KMeans(n_clusters=4, init="k-means++", random_state=RANDOM_SEED)
kmeans.fit(x)
kmeans.inertia_

plot_clusters(kmeans, x, axLabels=["Age", "Spending Score"])

#valutiamo altro modello...
x = df[["Annual Income (k$)", "Spending Score (1-100)"]].values
plot_ssd_curve(x) #si vede k=5
kmeans = KMeans(n_clusters=5, init="k-means++", random_state=RANDOM_SEED)
kmeans.fit(x)
kmeans.inertia_ # non si può confrontare con l'altra poichè relativa alla distanza dal centroide e la scala x è diversa, non è standardizzata
plot_clusters(kmeans, x, axLabels=["Annual Income (k$)", "Spending Score (1-100)"])

#valutiamo altro modello con tre variabili, va rappresentato in 3d...
x = df[["Age", "Annual Income (k$)", "Spending Score (1-100)"]].values
plot_ssd_curve(x) #si vede k=6
kmeans = KMeans(n_clusters=6, init="k-means++", random_state=RANDOM_SEED)
kmeans.fit(x)
kmeans.inertia_ # non si può confrontare con l'altra poichè relativa alla distanza dal centroide e la scala x è diversa, non è standardizzata

from mpl_toolkits import mplot3d
%matplotlib notebook
def plot_clusters3(model, data, axLables=None):
  centers = model.cluster_centers_
  y_kmean = model.predict(x)
  ax = plt.axes(projection="3d")
  ax.scatter3D(data[:,0], data[:,1], data[:,2], c=y_kmean)
  ax.scatter3D(centers[:,0], centers[:,1], centers[:,2], color="red")

plot_clusters3(kmeans, x)

#utilizziamo il modello
df_pred = pd.read_csv("https://raw.githubusercontent.com/ProfAI/machine-learning-fondamenti/main/datasets/mall_customers_predict.csv")
df_pred

x=df_pred[["Age", "Annual Income (k$)", "Spending Score (1-100)"]]
x.shape

y_kmeans = kmeans.predict(x)
y_kmeans #1° osservazione al secondo cluster, ecc

df_result = pd.DataFrame({
    "CustomerID" : df_pred["CustomerID"],
    "Customer Group": y_kmeans
})
df_result.head()

df_result.to_excel("mall_customers_prediction.xlsx")


,CustomerID,Customer Group
0,1,2
1,2,1
2,3,3
3,4,1
4,5,2
